In [2]:
# async file io, aiohttp
#!pip install aiohttp

In [ ]:
from aiohttp import web

import asyncio

class SourceChangedEvent:
    def __init__(self, source):
        self.source = source
        self._event = asyncio.Event()

    async def wait(self):
        await self._event.wait()
    
    def source_executed(self, success, errors=None):
        """
        """
        self.success = success
        self.errors = errors
        self._event.set()
                

class Runner:
    def __init__(self):
        self.source_change_queue = asyncio.Queue()
        
    async def update_source(self, source, block=True):
        """
        Update source code, wait for main loop
        to attempt to run it then return the success
        (or not) of this.
        """
        event = SourceChangedEvent(source)
        await self.source_change_queue.put(event)
        if block:
            await event.wait()
            return event
    
    def _get_updated_source(self):
        """
        If source was updated externally, return a SourceChangedEvent
        otherwise return None.
        """
        # TODO: Handle multiple events, by taking the last
        #       one and informing others they have failed.
        try:
            return self.source_change_queue.get_nowait()
        except asyncio.QueueEmpty:
            return
    
    async def run(self):
        while True:
            await asyncio.sleep(.1)
            source_changed = self._get_updated_source()
            
            if source_changed:
                # TODO - run new source here
                source_changed.source_completed(True)
                self.source_change_queue.task_done()
            
runner = Runner()


class WebAPI:
    def __init__(self, runner):
        self.runner = runner
        
    async def upload_source(self, request):
        """
        Upload new source to shoebot, wait for
        main loop to run it and return status.
        """
        data = await request.post()

        source = data['file']

        filename = source.filename

        source_file = source.file
        source_content = source_file.read()

        source_changed = await self.runner.update_source(source_content)

        return web.json_response({"compiled": source_changed.success,
                             "ran": source_changed.success,
                             "errors": source_changed.errors or []})
    
    def routes(self):
        return [web.post("/source", self.upload_source)]

async def web_api_runner(host="localhost", post=8085):
    api = WebAPI(runner)
    app = web.Application()
    app.add_routes([*api.routes()])

    runner = web.AppRunner(app)
    await runner.setup()
    site = web.TCPSite(runner, host, port)
    await site.start()

async def hello():
    while True:
        print("hello")
        await asyncio.sleep(30)


#loop = asyncio.get_event_loop()
#loop.run_until_complete(runner())

async def main():
    await asyncio.gather(web_api_runner(), 
                         hello(),
                         runner.run(),
                         return_exceptions=True)
    
await main()